In [1]:
from typing import List
import ee
import eelib
from eelib import eefuncs, sf
from pipelines import plcfgs, eerfpl

In [2]:
cfg = plcfgs.DataCubeCfg()

In [3]:
viewport = ee.FeatureCollection.from_file(
    filename="../000-data/intersection_64.shp",
    driver='ESRI Shapefile'
).geometry()


In [4]:
def set_tile_id(element: ee.Image):
    row = ee.Number(element.get('row')).format('%03d')
    col = ee.Number(element.get('col')).format('%03d')
    tile_id = row.cat(col)
    return element.set('tileId', tile_id)

In [5]:
# base collection
datacube = ee.ImageCollection('projects/fpca-336015/assets/williston-cba')\
    .filterBounds(viewport).map(set_tile_id).select(
        selectors=cfg.src_bands,
        opt_names=cfg.dest_bands
    )


In [6]:
# Optical Composite Setup
datacube = datacube.mosaic()

# isolate the seasonal composites
spring_sel = datacube.select(cfg.band_prefix.get('spring'))
summer_sel = datacube.select(cfg.band_prefix.get('summer'))
fall_sel = datacube.select(cfg.band_prefix.get('fall'))

seasonal_images = [spring_sel, summer_sel, fall_sel]
src_bands = [str(_.name) for _ in eelib.bands.S2SR]
optical_images = [image.select(image.bandNames(), src_bands) for image in seasonal_images]


In [ ]:
# Batch Create optical Derivs

# will co - register the ones we can

In [7]:

targets = []

for season, dates in cfg.seasons.items():
    base = ee.ImageCollection.s1Collection().\
        filter('relativeOrbitNumber_start == 64').\
        filterDate(*dates).\
        filterBounds(viewport)
    target_dates = [ee.Date(_) for _ in dates]
    
    # calc mid point
    mid_point = eefuncs.get_mid_point(target_dates)
    
    # difference the images date against the mid point
    target_collection = eefuncs.days_from_mid(
        collection=base,
        midPoint = mid_point
    )
    def setprops(element):
        return element.set('datestr', element.date().format('YYYY-MM-dd'),
                           'season', season)
    # inseart season 
    target_list = target_collection.filterDate(*dates)\
        .map(setprops).map(eefuncs.insert_groupid)\
        .toList(target_collection.size())
    target_list = target_list.map(lambda x: ee.Image(x))
    targets.append(target_list)



In [8]:
from pprint import pprint

pprint(targets[0].size().getInfo())

ee_list = ee.List(targets).flatten()
ee_list.size().getInfo()

12


23

In [9]:
col = ee.ImageCollection(ee_list)
col.size().getInfo()
pprint(col.first().getInfo())

{'bands': [{'crs': 'EPSG:32610',
            'crs_transform': [10,
                              0,
                              392202.90336598887,
                              0,
                              -10,
                              6081906.960594987],
            'data_type': {'precision': 'double', 'type': 'PixelType'},
            'dimensions': [29346, 21508],
            'id': 'VV'},
           {'crs': 'EPSG:32610',
            'crs_transform': [10,
                              0,
                              392202.90336598887,
                              0,
                              -10,
                              6081906.960594987],
            'data_type': {'precision': 'double', 'type': 'PixelType'},
            'dimensions': [29346, 21508],
            'id': 'VH'},
           {'crs': 'EPSG:32610',
            'crs_transform': [12795.92113506468,
                              -4053.747985428199,
                              427994.4226686542,
       

In [10]:

for season in cfg.seasons.keys():
    base = col.filter(f'season == "{season}"')
    
    dfms = base.aggregate_array('dfm').distinct().getInfo()
    print("Season: ")
    print("dfm, len")
    for dfm in dfms:
        query = base.filter(f'dfm == {dfm}')
        print(f"{dfm}, {query.size().getInfo()}")

Season: 
dfm, len
21, 3
9, 3
2, 3
14, 3
Season: 
dfm, len
28, 3
16, 3
4, 2
Season: 
dfm, len
15, 3


In [11]:
groupids = col.aggregate_array('groupid').distinct().getInfo()
groupids

['64_-122.36',
 '64_-122.83',
 '64_-123.37',
 '64_-122.47',
 '64_-122.94',
 '64_-123.33']

In [16]:
querys = [('spring', 2), ('summer', 16), ('fall', 15)] 
s1_imgs = [col.filter(f"season == '{q[0]}' && dfm == {q[1]}")
           for q in querys]
# s1_imgs = eefuncs.batch_despeckle(s1_imgs, sf.Boxcar(1))


In [18]:
s1_imgs[0].aggregate_array('system:index').getInfo()

['S1B_IW_GRDH_1SDV_20180609T015452_20180609T015517_011290_014BA0_39FD',
 'S1B_IW_GRDH_1SDV_20180609T015517_20180609T015542_011290_014BA0_2658',
 'S1B_IW_GRDH_1SDV_20180609T015542_20180609T015616_011290_014BA0_EE30']

In [24]:
s1_strs = [s1_img.aggregate_array('system:index').getInfo() 
           for s1_img in s1_imgs]
s1_strs_flatten = [f'{item}\n' for sublist in s1_strs for item in sublist]
s1_strs_flatten

['S1B_IW_GRDH_1SDV_20180609T015452_20180609T015517_011290_014BA0_39FD\n',
 'S1B_IW_GRDH_1SDV_20180609T015517_20180609T015542_011290_014BA0_2658\n',
 'S1B_IW_GRDH_1SDV_20180609T015542_20180609T015616_011290_014BA0_EE30\n',
 'S1B_IW_GRDH_1SDV_20180715T015454_20180715T015519_011815_015BE0_0362\n',
 'S1B_IW_GRDH_1SDV_20180715T015519_20180715T015544_011815_015BE0_D8D7\n',
 'S1B_IW_GRDH_1SDV_20180715T015544_20180715T015618_011815_015BE0_8287\n',
 'S1B_IW_GRDH_1SDV_20180913T015503_20180913T015528_012690_0176B4_78B3\n',
 'S1B_IW_GRDH_1SDV_20180913T015528_20180913T015553_012690_0176B4_0EB4\n',
 'S1B_IW_GRDH_1SDV_20180913T015553_20180913T015613_012690_0176B4_EB44\n']

In [25]:
with open('systems.txt', 'w+') as file:
    file.writelines(s1_strs_flatten)

In [17]:
import geemap

Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [19]:
Map.addLayer(s1_imgs[2], {"bands": ['VV'], 'min': -25, 'max': 10}, 'Fall')
Map.addLayer(s1_imgs[1], {"bands": ['VV'], 'min': -25, 'max': 10}, 'Summer')
Map.addLayer(s1_imgs[0], {"bands": ['VV'], 'min': -25, 'max': 10}, 'Spring')
Map.addLayer(viewport, {}, "Viewport")

Map.addLayerControl()